In [19]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
max_epochs = 4
batch_size = 4
l_r = 0.001


In [15]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])

In [16]:
train_dataset = torchvision.datasets.CIFAR10(download=True, root='./data', train=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(download=True, root='./data', train=False, transform=transform)

100%|██████████| 170498071/170498071 [00:09<00:00, 17668012.12it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [17]:
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

In [18]:
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [20]:
class ConvNet(nn.Module):
    def __init__(self,) -> None:
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [21]:
model = ConvNet().to(device)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=l_r)

In [27]:
n_total_steps = len(train_loader)
for e in range(max_epochs):
    for i,(img,lables) in enumerate(train_loader):

        images = img.to(device)
        lables = lables.to(device)

        op = model(images)
        l = loss(op, lables)

        optimizer.zero_grad()
        l.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{e+1}/{max_epochs}], Step [{i+1}/{n_total_steps}], Loss: {l.item():.4f}')

Epoch [1/4], Step [2000/12500], Loss: 1.5707
Epoch [1/4], Step [4000/12500], Loss: 1.3217
Epoch [1/4], Step [6000/12500], Loss: 1.4798
Epoch [1/4], Step [8000/12500], Loss: 0.6317
Epoch [1/4], Step [10000/12500], Loss: 0.6682
Epoch [1/4], Step [12000/12500], Loss: 1.3702
Epoch [2/4], Step [2000/12500], Loss: 1.3763
Epoch [2/4], Step [4000/12500], Loss: 1.6064
Epoch [2/4], Step [6000/12500], Loss: 1.7512
Epoch [2/4], Step [8000/12500], Loss: 1.2202
Epoch [2/4], Step [10000/12500], Loss: 1.2134
Epoch [2/4], Step [12000/12500], Loss: 1.4485
Epoch [3/4], Step [2000/12500], Loss: 2.2069
Epoch [3/4], Step [4000/12500], Loss: 1.3358
Epoch [3/4], Step [6000/12500], Loss: 1.8948
Epoch [3/4], Step [8000/12500], Loss: 0.9716
Epoch [3/4], Step [10000/12500], Loss: 0.9501
Epoch [3/4], Step [12000/12500], Loss: 1.2283
Epoch [4/4], Step [2000/12500], Loss: 1.2785
Epoch [4/4], Step [4000/12500], Loss: 0.2922
Epoch [4/4], Step [6000/12500], Loss: 1.2319
Epoch [4/4], Step [8000/12500], Loss: 0.6316
Epoc

In [29]:
with torch.no_grad():
    n_c = 0
    n_samples = 0
    
    n_c_corect = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]

    for images, lables in test_loader:
        images = images.to(device)
        lables = lables.to(device)
        op = model(images)

        _, pred = torch.max(op,1)

        n_samples+= lables.size(0)

        n_c+= (pred == lables).sum().item()

        for i in range(batch_size):
            label = lables[i]
            predicted = pred[i]
            if (label == predicted):
                n_c_corect[label]+=1
            n_class_samples[label]+=1

    acc = 100 * (n_c / n_samples)
    print(f'accuracy = {acc:.4f}')

    for i in range(10):
        acc = 100.0 * n_c_corect[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}:{acc}%')

accuracy = 58.9300
Accuracy of plane:69.5%
Accuracy of car:69.4%
Accuracy of bird:48.1%
Accuracy of cat:31.1%
Accuracy of deer:52.2%
Accuracy of dog:45.8%
Accuracy of frog:68.6%
Accuracy of horse:68.7%
Accuracy of ship:68.8%
Accuracy of truck:67.1%
